In [1]:
import MeCab as mc
import re
import urllib.request
import unicodedata
import pandas as pd

In [33]:
#テキストのクリーニング
def clean_text(text_string):
    text_string = re.sub(r'[０-９]', '', text_string)
    text_string = text_string.lower()
    return(text_string)

#slothlibのストップワードの取得
def get_stopword():
    slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    slothlib_file = urllib.request.urlopen(slothlib_path)
    slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
    slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
    return slothlib_stopwords

#ストップワードの追加
def add_stopword(slothlib_stopwords):
    s = open("Datas/stopwords.txt", "r", encoding="utf-8")
    stop = s.readlines()
    s.close()
    stop = [a.strip() for a in stop]
    slothlib_stopwords += stop
    return slothlib_stopwords

#ストップワードの除去
def except_stopwords(text):
    stopwords = get_stopword()
    ex_stopwords = add_stopword(stopwords)
    for i in text:
        for j in ex_stopwords:
            if i == j:
                text.remove(j)
    return text

#MeCabによる形態素解析
def extractter(text, flag):
    text = unicodedata.normalize("NFKC", text)
    tagger = mc.Tagger(r'-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    key= []
    word = []
    keyword =[]
    while node:
        nes = node.feature.split(",")
        wor = node.surface
        if wor != "":
            nes = node.feature.split(",")
            if flag == 0:
                if nes[0] == u"名詞":
                    if nes[6] == u"*":
                         keyword.append(wor)
                    else:
                        keyword.append(nes[6])
            else:
                if nes[0] == u"名詞":
                    if nes[6] == u"*":
                         keyword.append(wor)
                    else:
                        keyword.append(nes[6])
                elif nes[0] == u"動詞":
                    if nes[6] == u"*":
                        keyword.append(wor)
                    else:
                        keyword.append(nes[6])
                elif nes[0] == u"形容詞":
                    if nes[6] == u"*":
                        keyword.append(wor)
                    else:
                        keyword.append(nes[6])
                elif nes[0] == u"副詞":
                    if nes[6] == u"*":
                        keyword.append(wor)
                    else:
                        keyword.append(nes[6])
        node = node.next
        if node is None:
            break
    return keyword

#画像URLをあるかないかの2値に変換
def conversion_image(x):
    if x is not "0":
        return 1
    else:
        return 0

#テキストの結合
def join_data(text):
    texts = ' '.join(text)
    return texts

In [3]:
#メイン
#データの読み込み
data_input = pd.read_csv(filepath_or_buffer="Datas/all_data/all_data.csv", encoding="utf_8", sep=",")
print(len(data_input))

57926


In [4]:
data_input.head()

,screen_name,user_id,tweet_id,tweet,postdate,retweet,tag,image_url
0,岩手県宮古市公式アカウント,2.459551e+09,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01 00:00:00,0.0,0,0
1,花巻市,2.843129e+09,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01 00:00:00,0.0,0,0
2,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01 00:00:00,0.0,0,0
3,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01 00:00:00,0.0,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg
4,花巻市,2.843129e+09,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01 00:00:00,0.0,0,http://pbs.twimg.com/media/CVGe0BmUsAAdmrJ.png


In [5]:
data_input['image'] = data_input['image_url'].apply(conversion_image)
data_input.head()

,screen_name,user_id,tweet_id,tweet,postdate,retweet,tag,image_url,image
0,岩手県宮古市公式アカウント,2.459551e+09,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01 00:00:00,0.0,0,0,0
1,花巻市,2.843129e+09,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01 00:00:00,0.0,0,0,0
2,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01 00:00:00,0.0,0,0,0
3,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01 00:00:00,0.0,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1
4,花巻市,2.843129e+09,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01 00:00:00,0.0,0,http://pbs.twimg.com/media/CVGe0BmUsAAdmrJ.png,1


In [9]:
#postdateをdatetime形式に変換
data_input['postdate'] = pd.to_datetime(data_input['postdate'], format = '%Y-%m-%d')
data_input.head()

,screen_name,user_id,tweet_id,tweet,postdate,retweet,tag,image_url,image
0,岩手県宮古市公式アカウント,2.459551e+09,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01,0.0,0,0,0
1,花巻市,2.843129e+09,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01,0.0,0,0,0
2,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01,0.0,0,0,0
3,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1
4,花巻市,2.843129e+09,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGe0BmUsAAdmrJ.png,1


In [18]:
#テキストのクリーング
data_input['tweet'] = data_input['tweet'].apply(clean_text)
data_input.head(20)

,screen_name,user_id,tweet_id,tweet,postdate,retweet,tag,image_url,image,tweet2
0,岩手県宮古市公式アカウント,2.459551e+09,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01,0.0,0,0,0,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...
1,花巻市,2.843129e+09,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01,0.0,0,0,0,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...
2,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01,0.0,0,0,0,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...
3,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...
4,花巻市,2.843129e+09,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGe0BmUsAAdmrJ.png,1,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...
5,イーハトーブログ,1.968991e+08,6.714973e+17,きくっちです 今回ご紹介する は に併設されている です 久慈市 ランチ ...,2015-12-01,0.0,"駅1グルメ,久慈琥珀博物館,ビストロくんのこ,岩手県,久慈市,ランチ,グラタン,フランス料理",http://pbs.twimg.com/media/CVGjOXXUsAERZJa.jpg...,1,きくっちです 今回ご紹介する は に併設されている です 久慈市 ランチ ...
6,奥州市公式アカウント,2.644828e+08,6.715166e+17,今日から 月 クリスマスを前に 水沢図書館本館にクリスマス絵本コーナーができました 定...,2015-12-01,0.0,0,0,0,今日から 月 クリスマスを前に 水沢図書館本館にクリスマス絵本コーナーができました 定...
7,岩手県宮古市公式アカウント,2.459551e+09,6.715203e+17,献血にご協力をお願いします みなさんの善意の血液が 貴い命を救います,2015-12-01,0.0,0,0,0,献血にご協力をお願いします みなさんの善意の血液が 貴い命を救います
8,遠野市観光協会/Tono Tourist,2.324801e+09,6.715214e+17,和樂 月号 に遠野のことを載せてもらっています 年 日本美のルーツを田津ネル尋...,2015-12-01,0.0,"和樂,遠野,花巻,盛岡",0,0,和樂１ ２月号 に遠野のことを載せてもらっています 年 日本美のルーツを田津ネ...
9,花巻市,2.843129e+09,6.715248e+17,路線バス石鳥谷線のダイヤ改正について お知らせ 月 日 火曜日 より岩手県交通が運行...,2015-12-01,0.0,0,0,0,路線バス石鳥谷線のダイヤ改正について お知らせ 月 日 火曜日 より岩手県交通が運行...


In [19]:
#形態素解析
x = 0
data_input['tweet2'] = data_input['tweet2'].apply(lambda text:extractter(text, 0))
data_input.head(20)

,screen_name,user_id,tweet_id,tweet,postdate,retweet,tag,image_url,image,tweet2
0,岩手県宮古市公式アカウント,2.459551e+09,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01,0.0,0,0,0,"[広報, 月, 号, 市, ホームページ, アップ, 本号, 宮古, 港, 開港, 周年, ..."
1,花巻市,2.843129e+09,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01,0.0,0,0,0,"[自動, 証明, 写真, 機, 設置, 希望, 事業, 者, 募集, 申込, 受付, 期間,..."
2,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01,0.0,0,0,0,"[固定, 資産, 税, 各種, 申告, 等, 固定, 資産, 税, 毎年, 月, 日, 現在..."
3,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1,"[広報, 平成, 年, 月, 日, 号, 広報, まき, 月, 日, 号, 発行, 内容, ..."
4,花巻市,2.843129e+09,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGe0BmUsAAdmrJ.png,1,"[こども, 親子, 成島, 和紙, 凧, づくり, 体験, 参加, 者, 募集, 郷土, 成..."
5,イーハトーブログ,1.968991e+08,6.714973e+17,きくっちです 今回ご紹介する は に併設されている です 久慈市 ランチ ...,2015-12-01,0.0,"駅1グルメ,久慈琥珀博物館,ビストロくんのこ,岩手県,久慈市,ランチ,グラタン,フランス料理",http://pbs.twimg.com/media/CVGjOXXUsAERZJa.jpg...,1,"[今回, 紹介, 併設, 久慈, 市, ランチ, グラタン, フランス, 料理]"
6,奥州市公式アカウント,2.644828e+08,6.715166e+17,今日から 月 クリスマスを前に 水沢図書館本館にクリスマス絵本コーナーができました 定...,2015-12-01,0.0,0,0,0,"[今日, 月, クリスマス, 前, 水沢, 図書館, 本館, クリスマス, 絵本, コーナー..."
7,岩手県宮古市公式アカウント,2.459551e+09,6.715203e+17,献血にご協力をお願いします みなさんの善意の血液が 貴い命を救います,2015-12-01,0.0,0,0,0,"[献血, 協力, お願い, みなさん, 善意, 血液, 命]"
8,遠野市観光協会/Tono Tourist,2.324801e+09,6.715214e+17,和樂 月号 に遠野のことを載せてもらっています 年 日本美のルーツを田津ネル尋...,2015-12-01,0.0,"和樂,遠野,花巻,盛岡",0,0,"[和, 樂, 1, 2, 月, 号, 遠野, こと, 年, 日本, 美, ルーツ, 田津, ..."
9,花巻市,2.843129e+09,6.715248e+17,路線バス石鳥谷線のダイヤ改正について お知らせ 月 日 火曜日 より岩手県交通が運行...,2015-12-01,0.0,0,0,0,"[路線, バス, 石鳥谷, 線, ダイヤ, 改正, お知らせ, 月, 日, 火曜日, 岩手,..."


In [22]:
#ストップワードの除去
data_input['tweet2'] = data_input['tweet2'].apply(except_stopwords)
data_input.head(20)

,screen_name,user_id,tweet_id,tweet,postdate,retweet,tag,image_url,image,tweet2
0,岩手県宮古市公式アカウント,2.459551e+09,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01,0.0,0,0,0,"[広報, 号, ホームページ, アップ, 本号, 宮古, 港, 開港, 周年, 特集, 記事..."
1,花巻市,2.843129e+09,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01,0.0,0,0,0,"[自動, 証明, 写真, 機, 設置, 希望, 事業, 募集, 申込, 受付, 期間, 平成..."
2,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01,0.0,0,0,0,"[固定, 資産, 税, 各種, 申告, 固定, 資産, 税, 毎年, 日, 現在, 資産, ..."
3,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1,"[広報, 平成, 号, 広報, まき, 月, 日, 号, 発行, 内容, 花巻, ホームペー..."
4,花巻市,2.843129e+09,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGe0BmUsAAdmrJ.png,1,"[こども, 親子, 成島, 和紙, 凧, づくり, 体験, 参加, 募集, 郷土, 成島, ..."
5,イーハトーブログ,1.968991e+08,6.714973e+17,きくっちです 今回ご紹介する は に併設されている です 久慈市 ランチ ...,2015-12-01,0.0,"駅1グルメ,久慈琥珀博物館,ビストロくんのこ,岩手県,久慈市,ランチ,グラタン,フランス料理",http://pbs.twimg.com/media/CVGjOXXUsAERZJa.jpg...,1,"[紹介, 併設, 久慈, ランチ, グラタン, フランス, 料理]"
6,奥州市公式アカウント,2.644828e+08,6.715166e+17,今日から 月 クリスマスを前に 水沢図書館本館にクリスマス絵本コーナーができました 定...,2015-12-01,0.0,0,0,0,"[今日, クリスマス, 水沢, 図書館, 本館, クリスマス, 絵本, コーナー, 定番, ..."
7,岩手県宮古市公式アカウント,2.459551e+09,6.715203e+17,献血にご協力をお願いします みなさんの善意の血液が 貴い命を救います,2015-12-01,0.0,0,0,0,"[献血, 協力, お願い, 善意, 血液, 命]"
8,遠野市観光協会/Tono Tourist,2.324801e+09,6.715214e+17,和樂 月号 に遠野のことを載せてもらっています 年 日本美のルーツを田津ネル尋...,2015-12-01,0.0,"和樂,遠野,花巻,盛岡",0,0,"[和, 樂, 1, 2, 号, 遠野, 年, 日本, 美, ルーツ, 田津, ネル, 旅, ..."
9,花巻市,2.843129e+09,6.715248e+17,路線バス石鳥谷線のダイヤ改正について お知らせ 月 日 火曜日 より岩手県交通が運行...,2015-12-01,0.0,0,0,0,"[路線, バス, 石鳥谷, ダイヤ, 改正, お知らせ, 日, 火曜日, 岩手, 交通, 運..."


In [38]:
#リストの連結
data_input['tweet2'] = data_input['tweet2'].apply(join_data)
data_input.head(20)

,screen_name,user_id,tweet_id,tweet,postdate,retweet,tag,image_url,image,tweet2
0,岩手県宮古市公式アカウント,2.459551e+09,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01,0.0,0,0,0,広 報 号 ホ ー ム ペ ー ジ ア ッ プ 本 号 宮 古 港 ...
1,花巻市,2.843129e+09,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01,0.0,0,0,0,自 動 証 明 写 真 機 設 置 希 望 事 業 募 集 ...
2,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01,0.0,0,0,0,固 定 資 産 税 各 種 申 告 固 定 資 産 税 毎 ...
3,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1,広 報 平 成 号 広 報 ま き 月 日 号 発 行 ...
4,花巻市,2.843129e+09,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01,0.0,0,http://pbs.twimg.com/media/CVGe0BmUsAAdmrJ.png,1,こ ど も 親 子 成 島 和 紙 凧 づ く り 体 験 参 ...
5,イーハトーブログ,1.968991e+08,6.714973e+17,きくっちです 今回ご紹介する は に併設されている です 久慈市 ランチ ...,2015-12-01,0.0,"駅1グルメ,久慈琥珀博物館,ビストロくんのこ,岩手県,久慈市,ランチ,グラタン,フランス料理",http://pbs.twimg.com/media/CVGjOXXUsAERZJa.jpg...,1,紹 介 併 設 久 慈 ラ ン チ グ ラ タ ン フ ラ ン ス 料 理
6,奥州市公式アカウント,2.644828e+08,6.715166e+17,今日から 月 クリスマスを前に 水沢図書館本館にクリスマス絵本コーナーができました 定...,2015-12-01,0.0,0,0,0,今 日 ク リ ス マ ス 水 沢 図 書 館 本 館 ク リ ス マ ...
7,岩手県宮古市公式アカウント,2.459551e+09,6.715203e+17,献血にご協力をお願いします みなさんの善意の血液が 貴い命を救います,2015-12-01,0.0,0,0,0,献 血 協 力 お 願 い 善 意 血 液 命
8,遠野市観光協会/Tono Tourist,2.324801e+09,6.715214e+17,和樂 月号 に遠野のことを載せてもらっています 年 日本美のルーツを田津ネル尋...,2015-12-01,0.0,"和樂,遠野,花巻,盛岡",0,0,和 樂 1 2 号 遠 野 年 日 本 美 ル ー ツ ...
9,花巻市,2.843129e+09,6.715248e+17,路線バス石鳥谷線のダイヤ改正について お知らせ 月 日 火曜日 より岩手県交通が運行...,2015-12-01,0.0,0,0,0,路 線 バ ス 石 鳥 谷 ダ イ ヤ 改 正 お 知 ら せ 日 ...
